In [528]:
import matplotlib.pyplot as plt
import vtk
import numpy as np
import sys
import math
import os
import glob
from vtk.util.numpy_support import *
import pandas as pd
from multiprocessing import Pool
from vtk.util import numpy_support
from sklearn.decomposition import PCA
from numpy import linalg as LA
from skimage.measure import EllipseModel
from matplotlib.patches import Ellipse
import pickle
from sklearn import manifold
from numpy import linalg as LA

In [498]:
def read_multiblock(filename):
    reader = vtk.vtkXMLMultiBlockDataReader()
    reader.SetFileName(filename)
    reader.Update()
    return reader.GetOutput()
    
def write_multiblock(filename,data):
    writer = vtk.vtkXMLMultiBlockDataWriter()
    writer.SetInputData(data)
    writer.SetFileName(filename)
    writer.Update()

def ensureUtf(s):
    try:
        if type(s) == unicode:
            return s.encode('utf8', 'ignore')
    except: 
        return str(s)
    
def read_vti(filename):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(filename)
    reader.Update()
    return reader.GetOutput()

def write_vti(filename,data):
    writer = vtk.vtkXMLImageDataWriter()
    writer.SetInputData(data)
    writer.SetFileName(filename)
    writer.Update()
    
def compute_3d_to_1d_map(x,y,z,dimx,dimy,dimz):
    return x + dimx*(y+dimy*z)    
    
def extract_target_feature(feature_volume,confidence_th,feature_id):
    
    feature_volume.GetPointData().SetActiveScalars('feature_similarity')
    thresholding = vtk.vtkThreshold()
    thresholding.ThresholdByUpper(confidence_th)
    thresholding.SetInputData(feature_volume)
    seg = vtk.vtkConnectivityFilter()
    seg.SetInputConnection(thresholding.GetOutputPort())
    seg.SetExtractionModeToAllRegions()
    seg.ColorRegionsOn()
    thresholding2 = vtk.vtkThreshold()
    thresholding2.SetInputConnection(seg.GetOutputPort())
    thresholding2.ThresholdBetween(feature_id,feature_id)
    thresholding2.Update()
    return thresholding2.GetOutput()

def compute_1d_indices(single_feature,dims,gbounds):
    
    oneD_indices = [] 
    
    for i in range(single_feature.GetNumberOfPoints()):
        pts = single_feature.GetPoint(i)
        xval = int(((pts[0] - gbounds[0])/(gbounds[1]-gbounds[0]))*dims[0])
        yval = int(((pts[1] - gbounds[2])/(gbounds[3]-gbounds[2]))*dims[1])
        zval = int(((pts[2] - gbounds[4])/(gbounds[5]-gbounds[4]))*dims[2])
        
        if xval > dims[0]-1:
            xval = dims[0]-1
        
        if yval > dims[1]-1:
            yval = dims[1]-1
            
        if zval > dims[2]-1:
            zval = dims[2]-1    
        
        index = compute_3d_to_1d_map(xval,yval,zval,dims[0],dims[1],dims[2])
        oneD_indices.append(index)
        
        if index > dims[0]*dims[1]*dims[2] or index < 0:
            print (xval,yval,zval)
    
    if len(oneD_indices) != single_feature.GetNumberOfPoints():
        print ('mismatch in mumber of points!!')
        
    return oneD_indices


def find_match(data,target_feature,confidence_th,dims,size_threshold,gbounds,pickle_data):
    
    num_features = data.GetNumberOfBlocks()
    
    matched = []
    for i in range(num_features):
        block = data.GetBlock(i)
        
        vtk_pts = block.GetPoints()
        num_feature_pts = vtk_pts.GetNumberOfPoints()
        
        oneD_indices = []
        for j in range(num_feature_pts):
            pts = vtk_pts.GetPoint(j)
            xval = int(((pts[0] - gbounds[0])/(gbounds[1]-gbounds[0]))*dims[0])
            yval = int(((pts[1] - gbounds[2])/(gbounds[3]-gbounds[2]))*dims[1])
            zval = int(((pts[2] - gbounds[4])/(gbounds[5]-gbounds[4]))*dims[2])

            if xval > dims[0]-1:
                xval = dims[0]-1

            if yval > dims[1]-1:
                yval = dims[1]-1

            if zval > dims[2]-1:
                zval = dims[2]-1    

            index = compute_3d_to_1d_map(xval,yval,zval,dims[0],dims[1],dims[2])
            oneD_indices.append(index)

            if index > dims[0]*dims[1]*dims[2] or index < 0:
                print (xval,yval,zval)
        
        intersection_cardinality = len(set.intersection(*[set(target_feature), set(oneD_indices)]))
        if intersection_cardinality > 0:
            matched.append([pickle_data[i][0],pickle_data[i][1],intersection_cardinality,oneD_indices])
            break
            
    return matched


def check_merge(feature_id_temp,target_feature_1d_pts,ctstep,inpfname,pickle_fname,confidence_th,size_threshold,dims,gbounds):
    
    ## load features to be tested
    current_data = read_multiblock(inpfname)
    #load pickle file
    ffname = ensureUtf(pickle_fname)
    fname = open(ffname, 'rb')
    data = pickle.load(fname, encoding='latin1')
    data = np.asarray(data)
    
    matched = find_match(current_data,target_feature_1d_pts,confidence_th,dims,size_threshold,gbounds,data)
    
    idx = int(matched[0][1])
    
    if len(matched) > 0 and int(feature_id_temp) != idx:
        print ('Merge found: tstep: ' + str(matched[0][0]) + ' ' + str(matched[0][0]-75) +  ' fid: ' +  str(matched[0][1]))
#     else:
#         print ('oppssss: ' + str(matched[0][1]) + ' ' + str(feature_id_temp))
    
    return 0

In [525]:
## Parameters
confidence_th = 0.92
size_threshold = 100
dims  = [128,16,128]
initT = 75
endT = 408

# feature_tstep = 193
# feature_id = 3
# feature_tstep = 120 #(in Paraview should subtract 75)
# feature_id = 13
feature_tstep = 75 #(in Paraview should subtract 75)
feature_id = 1

feature_path = '../out/mfix_local_grid/'
feature_fname = feature_path + 'slic_compare_' + str(feature_tstep) + '.vti'
print ('initial feature file to load: ' + feature_fname)

#data_path = '../out/mfix_local_grid/segmented_features/'
data_path = '../out/segmented_features/'
full_cdb_path = '../bubble_all.cdb/'
out_tracked_feature_cdb_path = '../out/feature_cdb/'

initial feature file to load: ../out/mfix_local_grid/slic_compare_110.vti


In [526]:
feature_volume = read_vti(feature_fname)
gbounds = feature_volume.GetBounds()
target_feature = extract_target_feature(feature_volume,confidence_th,feature_id)
target_feature_1d_pts_og = compute_1d_indices(target_feature,dims,gbounds)


target_feature_1d_pts = target_feature_1d_pts_og
feature_id_temp = feature_id

## Forward tracking

forward_tracking = []
for ii in range(feature_tstep+1,endT):
    ## load features to be tested
    inpfname = data_path + 'segmented_' + str(ii) + '.vtm'
    current_data = read_multiblock(inpfname)
    
    #load pickle file
    pickle_fname = data_path + 'feature_values_' + str(ii) + '.pickle'
    ffname = ensureUtf(pickle_fname)
    fname = open(ffname, 'rb')
    data = pickle.load(fname, encoding='latin1')
    data = np.asarray(data)
    
    matched = find_match(current_data,target_feature_1d_pts,confidence_th,dims,size_threshold,gbounds,data)
    
    if len(matched) > 0:
        #print ('Matched feature: tstep: ' + str(matched[0][0]) + ' ' + str(matched[0][0]-75) +  ' fid: ' +  str(matched[0][1]))
        ## update the target feature
        target_feature_1d_pts = matched[0][3]
        
        ### Check merge/split
        if ii > feature_tstep+1:
            ctstep = ii-1
            inpfname1 = data_path + 'segmented_' + str(ctstep) + '.vtm'
            pickle_fname1 = data_path + 'feature_values_' + str(ctstep) + '.pickle'
            retval = check_merge(feature_id_temp,target_feature_1d_pts,ctstep,inpfname1,pickle_fname1,confidence_th,size_threshold,dims,gbounds)
#         else:
#             ## process the initial timestep here
#             print ('here')
        
        feature_id_temp = matched[0][1]
        forward_tracking.append([matched[0][0],matched[0][1],matched[0][2]])
    else:
        print ('No match found for foreward tracking')
        break   

        
target_feature_1d_pts = target_feature_1d_pts_og
feature_id_temp = feature_id

## Backward tracking
backward_tracking = []
for jj in range(feature_tstep-1,initT,-1):
    
    ## load features to be tested
    inpfname = data_path + 'segmented_' + str(jj) + '.vtm'
    current_data = read_multiblock(inpfname)
    
    #load pickle file
    pickle_fname = data_path + 'feature_values_' + str(jj) + '.pickle'
    ffname = ensureUtf(pickle_fname)
    fname = open(ffname, 'rb')
    data = pickle.load(fname, encoding='latin1')
    data = np.asarray(data)
    
    matched = find_match(current_data,target_feature_1d_pts,confidence_th,dims,size_threshold,gbounds,data)
    
    if len(matched) > 0:
        #print ('Matched feature: tstep: ' + str(matched[0][0]) + ' ' + str(matched[0][0]-75) +  ' fid: ' +  str(matched[0][1]))
        ## update the target feature
        
        target_feature_1d_pts = matched[0][3]
        
        ### Check merge/split
        if jj < feature_tstep-1:
            
            ctstep = jj+1
            print ('testing for: ' + str(ctstep) + ' and ' + str(jj) + ' ' + str(feature_id_temp))
                   
            inpfname1 = data_path + 'segmented_' + str(ctstep) + '.vtm'
            pickle_fname1 = data_path + 'feature_values_' + str(ctstep) + '.pickle'
            
            retval = check_merge(feature_id_temp,target_feature_1d_pts,ctstep,inpfname1,pickle_fname1,confidence_th,size_threshold,dims,gbounds)
#         else:
#             ## process the initial timestep here
#             print ('here')
        
        feature_id_temp = matched[0][1]
        backward_tracking.append([matched[0][0],matched[0][1],matched[0][2]])
    else:
        print ('No match found for backward tracking')
        break

## first combine all tracking results
all_tracked_results = backward_tracking + forward_tracking
all_tracked_results.append([feature_tstep,feature_id,len(target_feature_1d_pts_og)])

## sort the results by time step number
all_tracked_results.sort(key = lambda all_tracked_results: all_tracked_results[0]) 
#print (all_tracked_results)



Merge found: tstep: 126.0 51.0 fid: 0.0
Merge found: tstep: 143.0 68.0 fid: 0.0
No match found for foreward tracking
testing for: 109 and 108 3.0
testing for: 108 and 107 2.0
Merge found: tstep: 108.0 33.0 fid: 1.0
testing for: 107 and 106 1.0
testing for: 106 and 105 1.0
testing for: 105 and 104 1.0
testing for: 104 and 103 1.0
testing for: 103 and 102 1.0
testing for: 102 and 101 1.0
testing for: 101 and 100 1.0
testing for: 100 and 99 1.0
testing for: 99 and 98 1.0
testing for: 98 and 97 4.0
testing for: 97 and 96 4.0
testing for: 96 and 95 7.0
testing for: 95 and 94 6.0
testing for: 94 and 93 4.0
testing for: 93 and 92 4.0
testing for: 92 and 91 4.0
No match found for backward tracking


In [527]:
## load the complete database and filter the target specific cinemadatabase

##load csv file in dataframe
fname = full_cdb_path+'data.csv'
df = pd.read_csv(fname)

filtered_data=[]
for i in range(len(all_tracked_results)):
    data = df[(df['time_step']==all_tracked_results[i][0]) & (df['feature_id']==all_tracked_results[i][1])]
    data = data.to_numpy()
    data_list=list(data)    
    filtered_data.append(data_list)

## strip off one dimension, no changes to the actual data
final_filtered_data_tmp=[]
for i in range(np.shape(filtered_data)[0]):
    final_filtered_data_tmp.append(filtered_data[i][0])

# filter unexpected feature (from the top most layer most of the time)
final_filtered_data = []
for i in range(np.shape(final_filtered_data_tmp)[0]-1):
    if final_filtered_data_tmp[i][6] < final_filtered_data_tmp[i+1][6]:
        final_filtered_data.append(final_filtered_data_tmp[i])
    else:
        final_filtered_data.append(final_filtered_data_tmp[i])
        #break
    
#print (np.shape(final_filtered_data_tmp))    
final_filtered_data = np.asarray(final_filtered_data)

## clean the old result
os.system('rm -rf ' + out_tracked_feature_cdb_path + '*')

# mode for the folder 
mode = 0o777
out_feature_cdb_fname = out_tracked_feature_cdb_path + 'bubble.cdb'
os.mkdir(out_feature_cdb_fname, mode)

images_path = out_feature_cdb_fname + '/images'
os.mkdir(images_path, mode)

df = pd.DataFrame(final_filtered_data, index=range(final_filtered_data.shape[0]),
                          columns=range(final_filtered_data.shape[1]))

# ## drop the columns that has df column ids from previous data frame and serial ids
df = df.drop(df.columns[[0]], axis=1)
print (df)

## Name the columns
df.rename(columns={1:'time_step'}, inplace=True)
df.rename(columns={2:'feature_id'}, inplace=True)
df.rename(columns={3:'aspect_ratio'}, inplace=True)
df.rename(columns={4:'eigen_ratio'}, inplace=True)
df.rename(columns={5:'volume'}, inplace=True)
df.rename(columns={6:'x_center'}, inplace=True)
df.rename(columns={7:'FILE'}, inplace=True)

## store into csv file
out_csv_path = '/Users/sdutta/Codes/cinema_explorer/data/bubble.cdb' + '/data.csv'
#out_csv_path = out_feature_cdb_fname + '/data.csv'
df.to_csv(out_csv_path,index=False)

print ('done tracking')

       1  2         3         4     5           6                        7
0     91  6  0.824661  0.908838   125  0.00250248   images/bubble_6_91.png
1     92  4  0.767992  0.885315   174  0.00309833   images/bubble_4_92.png
2     93  4   0.60971   0.92097   220  0.00375276   images/bubble_4_93.png
3     94  4  0.754823  0.932327   248  0.00433877   images/bubble_4_94.png
4     95  6  0.596419  0.954819   297  0.00525143   images/bubble_6_95.png
..   ... ..       ...       ...   ...         ...                      ...
100  191  4  0.997112  0.707344  3837    0.064026  images/bubble_4_191.png
101  192  3  0.972306  0.716295  4008   0.0652065  images/bubble_3_192.png
102  193  3   0.91949  0.723754  4123   0.0664912  images/bubble_3_193.png
103  194  3  0.955579  0.745644  4151    0.067945  images/bubble_3_194.png
104  195  5  0.941376  0.720621  4256   0.0697911  images/bubble_5_195.png

[105 rows x 7 columns]
done tracking
